In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataset import TensorDataset
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx

import networkx as nx # for visualizing graphs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from functions.load_data import load_data, MarielDataset, edges

##### Encoder Procedure
1. Create an embedding of the node features (H) using an MLP
2. Create a message to pass through the edges of the graph using the embedded node features (H) and another MLP
3. Aggregate the messages created in Step 2 for each node to update node features
4. Pass the updated node features through another MLP to get the "Pre-posterior"; the posterior then becomes the softmax of the pre-posterior

# Load data

In [ ]:
data = MarielDataset(seq_len=10, n_joints=53)
dataloader = DataLoader(data, batch_size=32, shuffle=False)


print("\nGenerated {:,} sequences of shape: {}".format(len(data), data[0]))

In [ ]:
data[0].num_nodes

In [ ]:
data[0].num_edges

In [ ]:
data[0].is_directed()

In [ ]:
data[0].edge_index

# Train with NNConv layer

In [ ]:
from functions.modules import MLP, NNConv

In [ ]:
in_channels = data.seq_len*data.n_dim*data.n_joints
out_channels = 10 # latent dimension
model = NNConv(in_channels=in_channels, 
               out_channels=out_channels, 
               nn=MLP(
                   data[0].num_edge_features,
#                    data.seq_len*data[0].num_edge_features, 
                   100, in_channels*out_channels))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
def train():
    losses = []
    model.train()
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = F.nll_loss(output, batch.y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses

In [ ]:
for epoch in range(10):
    train()

In [ ]:
14455/49

# Scratch work

In [ ]:
# class MLPEncoder(MessagePassing):
#     def __init__(self, n_timesteps, n_joints, n_dim, hidden_dim, latent_dim, edge_features):
#         super(MLPEncoder, self).__init__(aggr='add')
#         self.mlp = MLP(n_joints*n_dim, hidden_dim, latent_dim)

#     def forward(self, x, edge_index):
#         # x has shape [n_timesteps, n_joints*n_dim]
#         # edge_index has shape [2, edge_features=1]
#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

#     def message(self, x_i, x_j):
#         # x_i has shape [edge_features, n_joints*n_dim]
#         # x_j has shape [edge_features, n_joints*n_dim]
#         tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [edge_features=1, 2*n_joints*n_dim]
#         return self.mlp(tmp)

#     def update(self, aggr_out):
#         # aggr_out has shape [n_timesteps, latent_dim]
#         return aggr_out

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = MLPEncoder(n_timesteps = 49, 
#                    n_joints=53, 
#                    n_dim=3, 
#                    hidden_dim=32, 
#                    latent_dim=16, 
#                    edge_features=1).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
# losses = []
# model.train()
# for epoch in tqdm(range(200)):
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
#     losses.append(loss.item())
#     loss.backward()
#     optimizer.step()

In [ ]:
# import torch
# from torch.nn import Sequential as Seq, Linear, ReLU
# from torch_geometric.nn import MessagePassing

# class MLP_Encoder(nn.Module):
    
#     def __init__(self, in_dim_channel, out_dim_channel, hidden_dim):
#         super(MLP_Encoder, self).__init__()
        
# #         self.mlp1 = MLP(in_dim, hidden_dim, out_dim)
#         self.pass_mlp = MLP(in_dim, hidden_dim, out_dim)
#         self.pass1 = NNConv(in_dim_channel, out_dim_channel, self.pass_mlp)
# #         self.mlp2 = MLP(in_dim, hidden_dim, out_dim)
    
#     def forward(self, inputs : Data):
#         x = self.pass1(inputs.x, inputs.edge_index, inputs.edge_attr)
#         node_features = self.mlp2(inputs.x)
#         return x